<a href="https://colab.research.google.com/github/tylerle29/Memotron/blob/main/MEMoTron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1

fatal: destination path 'Llama-3.1-Nemotron-Nano-4B-v1.1' already exists and is not an empty directory.


In [2]:
!pip install -qU vllm

In [3]:
!nohup python -m vllm.entrypoints.openai.api_server \
  --model Llama-3.1-Nemotron-Nano-4B-v1.1 \
  --trust-remote-code \
  --seed 1 \
  --host "0.0.0.0" \
  --port 5000 \
  --served-model-name "Llama-Nemotron-Nano-4B-v1.1" \
  --tensor-parallel-size 1 \
  --max-model-len 50_000 \
  --gpu-memory-utilization 0.95 \
  --enforce-eager \
  --enable-auto-tool-choice \
  --tool-parser-plugin "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_toolcall_parser.py" \
  --tool-call-parser "llama_nemotron_json" \
  --chat-template "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_generic_tool_calling.jinja" \
  > vllm.log &

nohup: redirecting stderr to stdout


In [4]:
!while ! grep -q "Application startup complete" vllm.log; do tail -n 1 vllm.log; sleep 5; done

2025-11-09 12:15:26.610240: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(APIServer pid=10344) INFO 11-09 12:15:35 [model.py:1510] Using max model len 50000
^C


In [5]:
!pip install flask-cors
!pip install flask pyngrok

In [6]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
import re

def kym_search_image(query):
    url = f"https://knowyourmeme.com/search?q={quote(query)}"
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    # 1. find the first meme-page link that contains an image with data-image
    candidates = soup.select('a[href^="/memes/"]')
    page = None

    for a in candidates:
        img = a.select_one("img[data-image]")
        if img:
            page = a
            img_tag = img
            break

    if page is None:
        print("No match")
        return None

    # page URL
    page_url = "https://knowyourmeme.com" + page["href"]

    # high-quality image from data-image
    img_url = img_tag["data-image"]

    print("Page:", page_url)
    print("Image:", img_url)
    return (page_url, img_url)

def scrape_text_kym(url):

    # 1. Request webpage

    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    response.raise_for_status()



    # 2. Parse HTML

    soup = BeautifulSoup(response.text, "html.parser")

    # Sets up the sections we need to parse.

    sections = {}

    relevant_titles = [

    "About", "Origin", "Spread", "History", "Background", "Meaning", "Usage", "Development", "Overview", "External References"

    ]



    # Parses them.

    for h2 in soup.find_all("h2"):

        title = h2.get_text(strip=True)

        if title in relevant_titles:

            content = []



            # Walk through siblings until the next h2

            for sib in h2.next_siblings:

                if getattr(sib, "name", None) == "h2":

                    break

                if sib.name in ["p"]:

                    text = sib.get_text(" ", strip=True)

                    if text:
                        text = re.sub(r"\[.*?\]", "", text)
                        content.append(text.strip())



            sections[title] = "\n".join(content)



    return sections

def scrape_visible_text(url, timeout=10):
    r = requests.get(url, timeout=timeout, headers={
        "User-Agent": "Mozilla/5.0"
    })
    r.raise_for_status()
    html = r.text
    # remove all HTML tags
    text = re.sub(r"<[^>]+>", " ", html)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def scrape_all_paragraphs(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url, headers=headers)
  response.raise_for_status()

  soup = BeautifulSoup(response.text, "html.parser")

  paragraphs = []
  for p in soup.find_all("p"):
      text = p.get_text(" ", strip=True)
      if text:
          paragraphs.append(text)

  return "\n".join(paragraphs)



In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from openai import OpenAI
import os
from pyngrok import ngrok
import threading

app = Flask(__name__)
CORS(app)

conversation = ""
conv_lock = threading.Lock()

client = OpenAI(base_url="http://127.0.0.1:5000/v1", api_key="dummy")

@app.route("/api/respond", methods=["POST"])
def respond():
    global conversation
    data = request.json
    user_input = data.get("input", "")

    page_url, img_url = kym_search_image(user_input)
    sections = scrape_text_kym(page_url)

    context = ""
    for title, text in sections.items():
        context += f"{title}: {text}\n"

    buffer = context.strip()
    max_chars = 2500 + len(context)

    while len(buffer) < max_chars:
        prompt = (
            "Given the following context, propose 3–5 broad google search terms "
            "that would expand the context. Return ONLY a JSON array of strings.\n\n"
            f"CONTEXT:\n{buffer[:2000]}"
        )

        messages = [
            {"role": "system", "content": "Be precise. Output JSON only."},
            {"role": "user", "content": prompt}
        ]

        resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=messages,
            temperature=0.4,
            stream=False
        )

        raw_terms = resp.choices[0].message.content
        try:
            import json
            terms = json.loads(raw_terms)
            if not isinstance(terms, list):
                break
        except:
            break

        for term in terms:
            google_page = f"https://www.google.com/search?q={term.replace(' ', '+')}"
            scraped = scrape_visible_text(google_page)
            if scraped:
                buffer += "\n" + scraped
                if len(buffer) >= max_chars:
                    break

        compression_prompt = (
            "Summarize the important points of the following text. "
            "Be succinct and do not exceed 2 paragraphs.\n\n"
            f"{buffer[-3000:]}"
        )
        comp_resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=[
                {"role": "system", "content": "Summaries only."},
                {"role": "user", "content": compression_prompt}
            ],
            temperature=0.3,
            stream=False
        )

        summary = comp_resp.choices[0].message.content.strip()
        buffer = summary

        if len(buffer) >= max_chars:
            break

    final_prompt = (
        "Here is all the gathered context. Analyze the meme thoroughly.\n\n"
        f"{buffer[:max_chars]}"
    )

    final_resp = client.chat.completions.create(
        model="Llama-Nemotron-Nano-4B-v1.1",
        messages=[
            {"role": "system", "content": "Precise analysis."},
            {"role": "user", "content": final_prompt}
        ],
        temperature=0.6,
        stream=False
    )

    print(final_prompt)
    output = final_resp.choices[0].message.content

    # safe append
    with conv_lock:
        conversation += "This is the intial input: " + final_prompt + "\n"
        conversation += "This is the intial output: " + output + "\n"

    return jsonify({"input": user_input, "output": output})


@app.route("/api/converse", methods=["POST"])
def converse():
    global conversation
    data = request.json
    user_input = data.get("input", "")

    with conv_lock:
        prompt_context = conversation
        conversation += "This is added inputs by the user: " + user_input + "\n"

    prompt = (
        "Hopefully, make a 2-3 consentences. "
        "This is the most recent input (Focus on this): " + user_input + "\n"
        f"CONTEXT:\n{prompt_context}"
    )

    messages = [
        {"role": "system", "content": "Be precise. Output JSON only."},
        {"role": "user", "content": prompt}
    ]

    resp = client.chat.completions.create(
        model="Llama-Nemotron-Nano-4B-v1.1",
        messages=messages,
        temperature=0.4,
        stream=False
    )

    output = resp.choices[0].message.content

    with conv_lock:
        conversation += "This is the added output: " + output + "\n"

    return jsonify({"input": user_input, "output": output})


@app.route("/api/search_image", methods=["POST"])
def search_image():
    data = request.json
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "query field required"}), 400

    img_url = kym_search_image(query)
    if img_url is None:
        return jsonify({"query": query, "image_url": None, "status": "not_found"})

    return jsonify({"query": query, "image_url": img_url})


NGROK_AUTH_TOKEN = "34oJGiQHYcc8THKsPzsqUXBLMi6_5oBmkk9dHcsN2T2S5W2gG"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5001).public_url
print("Public URL:", public_url)

app.run(port=5001)


Public URL: https://nonrealistic-ungrimed-luvenia.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:16:20] "OPTIONS /api/respond HTTP/1.1" 200 -


Page: https://knowyourmeme.com/memes/my-mom-is-kinda-homeless-please-speed-i-need-this
Image: https://i.kym-cdn.com/entries/icons/original/000/055/583/my-mom-is-kinda-homless.jpg


INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:17:09] "POST /api/respond HTTP/1.1" 200 -


Here is all the gathered context. Analyze the meme thoroughly.

About: My Mom is Kinda Homeless and Please Speed I Need This , also known as My Mom is Kind of Homeless , refer to quotes from the IShowSpeed Trying Not To Laugh clip, said by an emotional kid trying to get streamer IShowSpeed (or Speed) to throw a Fortnite game in his " EARLY STREAM! " livestream from late December 2021. Speed is seen trying not to laugh at the kid's pleading, and is then seen smiling when another player calls him out, saying, "Speed, I'm watching your stream, why are you trying not to laugh, bruh?" The clip went notably viral when it was used as a reaction GIF and in video caption memes in 2023. The kid's quote about his mom being kind of homeless gained its own memetic spread as a catchphrase and phrasal template in the following years.
In late October 2025, IShowSpeed recreated the My Mom is Kinda Homeless clip in a "reunion" stream with the two guys in the original clip. The kid who said, "My mom is k

INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:18:23] "OPTIONS /api/respond HTTP/1.1" 200 -


Page: https://knowyourmeme.com/memes/my-mom-is-kinda-homeless-please-speed-i-need-this
Image: https://i.kym-cdn.com/entries/icons/original/000/055/583/my-mom-is-kinda-homless.jpg


INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:19:05] "POST /api/respond HTTP/1.1" 200 -


Here is all the gathered context. Analyze the meme thoroughly.

About: My Mom is Kinda Homeless and Please Speed I Need This , also known as My Mom is Kind of Homeless , refer to quotes from the IShowSpeed Trying Not To Laugh clip, said by an emotional kid trying to get streamer IShowSpeed (or Speed) to throw a Fortnite game in his " EARLY STREAM! " livestream from late December 2021. Speed is seen trying not to laugh at the kid's pleading, and is then seen smiling when another player calls him out, saying, "Speed, I'm watching your stream, why are you trying not to laugh, bruh?" The clip went notably viral when it was used as a reaction GIF and in video caption memes in 2023. The kid's quote about his mom being kind of homeless gained its own memetic spread as a catchphrase and phrasal template in the following years.
In late October 2025, IShowSpeed recreated the My Mom is Kinda Homeless clip in a "reunion" stream with the two guys in the original clip. The kid who said, "My mom is k

INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:19:28] "OPTIONS /api/converse HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 12:19:34] "POST /api/converse HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from openai import OpenAI
import os
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)
conversation = ""
client = OpenAI(base_url="http://127.0.0.1:5000/v1", api_key="dummy")

@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    user_input = data.get("input", "")

    # initial context from KYM
    page_url, img_url = kym_search_image(user_input)
    sections = scrape_text_kym(page_url)

    context = ""
    for title, text in sections.items():
        context += f"{title}: {text}\n"

    # running text buffer
    buffer = context.strip()
    max_chars = 2500 + len(context)

    # loop until buffer fills
    while len(buffer) < max_chars:
        # ask model for new search terms
        prompt = (
            "Given the following context, propose 3–5 broad google search terms "
            "that would expand the context. Return ONLY a JSON array of strings.\n\n"
            f"CONTEXT:\n{buffer[:2000]}"
        )

        messages = [
            {"role": "system", "content": "Be precise. Output JSON only."},
            {"role": "user", "content": prompt}
        ]

        resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=messages,
            temperature=0.4,
            stream=False
        )

        raw_terms = resp.choices[0].message.content
        try:
            import json
            terms = json.loads(raw_terms)
            if not isinstance(terms, list):
                break
        except:
            break

        # scrape google summaries for each term
        for term in terms:
            google_page = f"https://www.google.com/search?q={term.replace(' ', '+')}"
            scraped = scrape_visible_text(google_page)
            if scraped:
                buffer += "\n" + scraped
                if len(buffer) >= max_chars:
                    break

        # compress via model
        compression_prompt = (
            "Summarize the important points of the following text. "
            "Be succinct and do not exceed 2 paragraphs.\n\n"
            f"{buffer[-3000:]}"
        )
        comp_resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=[
                {"role": "system", "content": "Summaries only."},
                {"role": "user", "content": compression_prompt}
            ],
            temperature=0.3,
            stream=False
        )

        summary = comp_resp.choices[0].message.content.strip()
        buffer = summary  # next loop uses summary as new context

        if len(buffer) >= max_chars:
            break

    # final answer request
    final_prompt = (
        "Here is all the gathered context. Analyze the meme thoroughly.\n\n"
        f"{buffer[:max_chars]}"
    )

    final_resp = client.chat.completions.create(
        model="Llama-Nemotron-Nano-4B-v1.1",
        messages=[
            {"role": "system", "content": "Precise analysis."},
            {"role": "user", "content": final_prompt}
        ],
        temperature=0.6,
        stream=False
    )
    print(final_prompt)

    conversation += "This is the intial input: " + final_prompt + "\n";
    output = final_resp.choices[0].message.content
    conversation += "This is the intial output: " + output + "\n";

    return jsonify({"input": user_input, "output": output})

@app.route("/api/converse", methods=["POST"])
def converse():
  data = request.json
  user_input = data.get("input", "")
  prompt = (
    "Given the following context, give a response to the most recent input. Hopefully, 2-3 sentences. "
    "This is the most recent input (Focus on this): " + user_input + "\n"
    f"CONTEXT:\n{conversation}"
        )

  conversation += "This is added inputs by the user: " + user_input + "\n";


  messages = [
      {"role": "system", "content": "Be precise. Output JSON only."},
      {"role": "user", "content": prompt}
  ]

  resp = client.chat.completions.create(
      model="Llama-Nemotron-Nano-4B-v1.1",
      messages=messages,
      temperature=0.4,
      stream=False
  )

  output = resp.choices[0].message.content

  conversation += "This is the added output: " + output + "\n";

  return jsonify({"input": user_input, "output": output})

@app.route("/api/search_image", methods=["POST"])
def search_image():
    data = request.json
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "query field required"}), 400

    img_url = kym_search_image(query)

    if img_url is None:
        return jsonify({"query": query, "image_url": None, "status": "not_found"})

    return jsonify({"query": query, "image_url": img_url})

NGROK_AUTH_TOKEN = "34oJGiQHYcc8THKsPzsqUXBLMi6_5oBmkk9dHcsN2T2S5W2gG"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5001).public_url
print("Public URL:", public_url)

# -------------------
# Run server
# -------------------
app.run(port=5001)